# Introduction

This notebook introduces the library SPAN, a small library designed to make using setools 4 simple in a Jupyter notebook such as this one.

Jupyter notebooks are an interactive environment that let's us write text (in Markdown) and code together. What's powerful is that the code is executable (unless you are viewing this on the web in a read-only mode). That let's you write queries and text together at the same time. You can get a feel for what's possible in this awesome notebook on [Regex Golf from XKCD](http://nbviewer.jupyter.org/url/norvig.com/ipython/xkcd1313.ipynb). There is also the more official (and boring) [introduction](https://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/).

Using setools within Jupyter notebook is an amazingly productive way to do policy analysis. It becomes simple to keep notes alongside any queries you do or, almost more importantly, write simple scripts that allow you to do more powerful policy analysis.

To get started, let's import the library and load a Fedora 25 binary and source policy as an example:

In [1]:
# Import span - this complication is just to handle running this in the development tree
try:
    import span as se
except:
    import os
    path = os.path.dirname(os.getcwd())
    import sys
    sys.path.insert(0, path)
    import span as se
p = se.load_policy("fedora-25-policy.30")
ps = se.load_refpolicy_source("serefpolicy-fedora-25")

# Example - Protecting Passwords

We'll get to the details of how to use the library soon. But first, let's start with an example to demonstrate some of the power that we get from this environment.

Let's do that by answering a common security question: what applications can write to the shadow file and are any of those applications controllable by users?

But first, did I mention that we can include images in these notebooks?

<img src="https://i.imgur.com/D5LidQ1.jpg">

## Domains That Can Write /etc/shadow

Anyway, for the first part of that question, we can do a simple search for rules using the method terules_query:

In [2]:
passwd_writers = p.terules_query(target="shadow_t", tclass=["file"], perms=["write", "append", "relabelto"])
passwd_writers

,source,target,tclass,perms,cond
0,cockpit_session_t,shadow_t,file,"(append, unlink, open, write, setattr, create, getattr, ioctl, rename, read, link, lock)",None
1,files_unconfined_type,file_type,file,"(unlink, execute, quotaon, execute_no_trans, swapon, link, getattr, append, ioctl, rename, audit_access, lock, write, setattr, read, relabelto, relabelfrom, create, open, mounton)",None
2,groupadd_t,shadow_t,file,"(append, unlink, open, write, setattr, relabelto, relabelfrom, create, getattr, ioctl, rename, read, link, lock)",None
3,passwd_t,shadow_t,file,"(append, unlink, open, write, setattr, relabelto, relabelfrom, create, getattr, ioctl, rename, read, link, lock)",None
4,pegasus_openlmi_account_t,shadow_t,file,"(append, unlink, open, write, setattr, relabelto, relabelfrom, create, getattr, ioctl, rename, read, link, lock)",None
5,rpm_script_t,file_type,file,"(append, unlink, open, write, setattr, relabelto, relabelfrom, create, getattr, ioctl, rename, read, link, lock)",None
6,rpm_script_t,shadow_t,file,(relabelto),None
7,rpm_t,file_type,file,"(append, unlink, open, write, setattr, relabelto, relabelfrom, create, getattr, ioctl, rename, read, link, lock)",None
8,rpm_t,shadow_t,file,(relabelto),None
9,sandbox_domain,sandbox_typeattr_1,file,"(append, write, getattr, ioctl, read, lock)",None


A few things to note about this if you are new to Jupyter notebook. First, by default Jupyter will display the output from the last expression, which is why just putting the variable we assigned the results to on it's own line caused the display. If we didn't need to use the output later we could have just omitted assigning the output to a variable.

Next, you'll notice that the output is a nicely formatted table. The results are actually in a Pandas [DataFrame](http://pandas.pydata.org/pandas-docs/stable/dsintro.html). [Pandas](http://pandas.pydata.org) is a common and very powerful data analysis tool for Python. Here it lets us display the data nicely, including allowing sorting (try clicking on the column titles to sort by that column).

It also let's us further search. For example, let's see which domains where allowed access directly to `shadow_t` rather than to an attribute that includes `shadow_t`.

In [3]:
passwd_writers[passwd_writers.target == "shadow_t"]

,source,target,tclass,perms,cond
0,cockpit_session_t,shadow_t,file,"(append, unlink, open, write, setattr, create, getattr, ioctl, rename, read, link, lock)",None
2,groupadd_t,shadow_t,file,"(append, unlink, open, write, setattr, relabelto, relabelfrom, create, getattr, ioctl, rename, read, link, lock)",None
3,passwd_t,shadow_t,file,"(append, unlink, open, write, setattr, relabelto, relabelfrom, create, getattr, ioctl, rename, read, link, lock)",None
4,pegasus_openlmi_account_t,shadow_t,file,"(append, unlink, open, write, setattr, relabelto, relabelfrom, create, getattr, ioctl, rename, read, link, lock)",None
6,rpm_script_t,shadow_t,file,(relabelto),None
8,rpm_t,shadow_t,file,(relabelto),None
11,secadm_t,shadow_t,file,"(relabelto, relabelfrom, getattr)",None
13,setfiles_mac_t,shadow_t,file,(relabelto),None
15,setfiles_t,shadow_t,file,(relabelto),None
16,sysadm_passwd_t,shadow_t,file,"(append, unlink, open, write, setattr, relabelto, relabelfrom, create, getattr, ioctl, rename, read, link, lock)",None


Just to make certain this is the right type, let's check the filecontexts from the source:

In [4]:
print(ps.file_contexts("shadow_t"))

./system/authlogin.fc:11:/etc/gshadow.*		--	gen_context(system_u:object_r:shadow_t,s0)
./system/authlogin.fc:12:/etc/nshadow.*		--	gen_context(system_u:object_r:shadow_t,s0)
./system/authlogin.fc:13:/etc/shadow.*		--	gen_context(system_u:object_r:shadow_t,s0)
./system/authlogin.fc:14:/etc/security/opasswd	--	gen_context(system_u:object_r:shadow_t,s0)
./system/authlogin.fc:15:/etc/security/opasswd\.old	--	gen_context(system_u:object_r:shadow_t,s0)
./system/authlogin.fc:55:/var/db/shadow.*	--	gen_context(system_u:object_r:shadow_t,s0)



That looks right . . . I guess. I wonder what the heck `/etc/security/opasswd` is??!? Oh well, it covers good, old-fashioned `/etc/shadow`, so let's move on.

## Finding Domain Transitions from Login Domains

Now we have all of the domains that can write to shadow, so let's answer the second part by determining whether any of the login domains can transition to these domains. Just to keep this short, I'm going to just check for the 3 standard login types (normally you would need to figure out whether there were more).

So let's make a list of those types:

In [5]:
login_domains = ["user_t", "sysadm_t", "secadm_t"]

# Just verify that these are in the policy
for domain in login_domains:
    print(p.lookup_type(domain))

user_t
sysadm_t
secadm_t


Now we can find the domain transitions that are allowed:

In [6]:
user_transitions = p.terules_query(source="user_t", tclass=["process"], perms=["transition"])
user_transitions

,source,target,tclass,perms,cond
0,domain,abrt_helper_t,process,(transition),None
1,user_t,bluetooth_helper_t,process,"(signull, sigstop, sigchld, sigkill, transition, getattr, signal)",None
2,user_t,cdrecord_t,process,"(signull, sigstop, sigchld, sigkill, transition, getattr, signal)",None
3,user_t,chfn_t,process,(transition),None
4,user_t,chkpwd_t,process,"(transition, getattr)",None
5,user_t,chrome_sandbox_t,process,"(signull, sigstop, sigchld, sigkill, transition, getattr, signal)",None
6,user_t,crontab_t,process,"(signull, sigstop, sigchld, sigkill, transition, getattr, signal)",None
7,user_t,exim_t,process,(transition),None
8,user_t,gconfd_t,process,"(signull, sigstop, sigchld, sigkill, transition, getattr, signal)",None
9,user_t,git_session_t,process,(transition),git_session_users


## Putting It Together - Accessible Domains That Can Write /etc/shadow
And now we can see if `user_t` is allowed to transition to any of the domains that can write shadow passwords. To do this, we are going to leverage the built-in sets from Python (which are fantastic). You can get a single column from a DataFrame with `DataFrame.column_name` and, because that is iterable, build a set from that. So we build a set from the targets of the user transitions and the source of the password writers.

Once you have the sets it's simple to perform set intersection (with the `&` operator) to find the types that are both sets.

In [7]:
user_passwd_writers = set(user_transitions.target) & set(passwd_writers.source)
user_passwd_writers

{<span.span.Type at 0x7f5458507dd0>,
 <span.span.Type at 0x7f5457914990>}

So, as expected, `passwd_t` `updpwd_t` can write to shadow and is accessible from `user_t`. `sandbox_domain` is more surprising.

Let's see what types are in that attribute.

In [8]:
p.types_in_attribute("sandbox_domain")

[<Type(<SELinuxPolicy("fedora-25-policy.30")>, "sandbox_t")>]

## Automating Checks for All Login Domains

Wait, you should be saying, we're only answering the question for one login domain! Let's build a simple function to do this for all of the login domains (because that nicely shows the power of having a full programming language right here for analysis).

In [9]:
# Yeah for stupidly long function names
def check_accessible_domains_that_can_write_shadow(login_domains):
    for login_domain in login_domains:
        # These are the same queries we did above, just using the passed in login domain types as appropriate.
        writers = p.terules_query(target="shadow_t", tclass=["file"], perms=["write", "append", "relabelto"])
        accessible_domains = p.terules_query(source=login_domain, tclass=["process"], perms=["transition"])
        ad_set = set(accessible_domains.target)
        # Add the login domain to see if it has direct access.
        #
        # Since we are doing comparisons it must be the object here and _not_ the
        # string for the type. These kinds of issues crop up occasionally so keep an eye out for them.
        ad_set.add(p.lookup_type(login_domain))
        print("Shadow writers accessible by " + login_domain + ":")
        print(ad_set & set(writers.source))
        
# Notice how I'm referring to a list that we created way up the page?
check_accessible_domains_that_can_write_shadow(login_domains)

Shadow writers accessible by user_t:
{<span.span.Type object at 0x7f545763c090>, <span.span.TypeAttribute object at 0x7f545763ef50>, <span.span.Type object at 0x7f545763d390>}
Shadow writers accessible by sysadm_t:
{<span.span.TypeAttribute object at 0x7f545763f710>, <span.span.Type object at 0x7f545762d7d0>, <span.span.Type object at 0x7f545763eb90>, <span.span.Type object at 0x7f545763fb50>, <span.span.Type object at 0x7f545763e650>, <span.span.Type object at 0x7f545762da90>, <span.span.Type object at 0x7f545763f290>, <span.span.Type object at 0x7f545762d410>, <span.span.Type object at 0x7f545762db50>, <span.span.Type object at 0x7f545762d1d0>}
Shadow writers accessible by secadm_t:
{<span.span.Type object at 0x7f545764a550>, <span.span.Type object at 0x7f545764a810>, <span.span.Type object at 0x7f545764bf10>}


Look at that - both `secadm_t` and `sysadm_t` have direct access. Let's see what that looks like.

In [10]:
passwd_writers[passwd_writers.source == "sysadm_t"]

,source,target,tclass,perms,cond
17,sysadm_t,file_type,file,"(relabelto, relabelfrom, getattr)",None
18,sysadm_t,shadow_t,file,"(relabelto, relabelfrom, getattr)",None


And the moral of the story, kids, is never forget about broad relabeling privileges. Being able to relabel a file to a type has the same security implications as writing the same type. Though, come to think of it, you have to also be able to create a file in `/etc` with the correct name. Let's check that.

In [11]:
p.terules_query(source="sysadm_t", target="etc_t", perms=["add_name"])

,source,target,tclass,perms,cond
0,sysadm_t,etc_t,dir,"(open, write, search, add_name, getattr, ioctl, read, lock, remove_name)",None
1,sysadm_t,non_security_file_type,dir,"(reparent, unlink, open, write, setattr, relabelto, search, relabelfrom, rmdir, add_name, create, getattr, ioctl, rename, read, link, lock, remove_name)",None


## Checking Automatic Transitions

Notice that we only checked for _allowed_ transitions. We didn't see if any where automatic. Let's do that now.

In [12]:
automatic_transitions = p.transrules_query(source="user_t", tclass=["process"])
automatic_transitions

,source,target,tclass,default,filename,cond
0,user_t,abrt_helper_exec_t,process,abrt_helper_t,None,None
1,user_t,bluetooth_helper_exec_t,process,bluetooth_helper_t,None,None
2,user_t,cdrecord_exec_t,process,cdrecord_t,None,None
3,user_t,chfn_exec_t,process,chfn_t,None,None
4,user_t,chkpwd_exec_t,process,chkpwd_t,None,None
5,user_t,chrome_sandbox_exec_t,process,chrome_sandbox_t,None,None
6,user_t,crontab_exec_t,process,crontab_t,None,None
7,user_t,dbusd_exec_t,process,user_dbusd_t,None,None
8,user_t,exim_exec_t,process,exim_t,None,None
9,user_t,fusermount_exec_t,process,mount_t,None,None


And using the same approach with sets, we can see if any of those transitions are automatic (just for `user_t` for now).

In [13]:
set(automatic_transitions.default) & user_passwd_writers

{<span.span.Type at 0x7f5458507dd0>, <span.span.Type at 0x7f5457914990>}

Hmmm - wasn't one of those an attribute? And the default type of a transition rule can't be an attribute.

Let's try that again, but expand attributes.

In [14]:
expanded_writers = p.expand_attributes(user_passwd_writers)
print("with attributes expanded: " + str(expanded_writers))
        
set(automatic_transitions.default) & expanded_writers

with attributes expanded: {<Type(<SELinuxPolicy("fedora-25-policy.30")>, "passwd_t")>, <Type(<SELinuxPolicy("fedora-25-policy.30")>, "sandbox_t")>, <Type(<SELinuxPolicy("fedora-25-policy.30")>, "updpwd_t")>}


{<Type(<SELinuxPolicy("fedora-25-policy.30")>, "passwd_t")>,
 <Type(<SELinuxPolicy("fedora-25-policy.30")>, "updpwd_t")>}

Well - doesn't look like that's an automatic transition (which isn't surprising). But I included this example to remind you to be careful about attributes. The rule searching will check attributes for you by default, but you have to be careful in your own code.

## Wrapping Up - Entrypoints and Userspace Checks

Two last things.

First - let's see the entrypoints for these domains. I'm including this because I _never_ get tired of bringing this up. It's critical to know what code runs in a domain because that's how you know whether you should trust that code with the access granted to the domain.

In [15]:
# This will handle the sandbox_domain attribute for us automatically in that the domain will be matched as well
# as any types with that attribute with rules explicitly referencing them.
p.terules_query(source=user_passwd_writers, tclass=["file"], perms=["entrypoint"])

,source,target,tclass,perms,cond
0,passwd_t,passwd_exec_t,file,"(open, execute, execute_no_trans, getattr, ioctl, read, lock, entrypoint)",None
1,sandbox_domain,exec_type,file,"(open, execute, execute_no_trans, getattr, ioctl, read, lock, entrypoint)",None
2,sandbox_domain,mountpoint,file,(entrypoint),None
3,updpwd_t,updpwd_exec_t,file,"(open, execute, execute_no_trans, getattr, ioctl, read, lock, entrypoint)",None


Well - sandbox_domain is certainly concerning. I'm certain that the magic of containers is all good though.

Let's check the file contexts for the normal ones.

In [16]:
print(ps.file_contexts("passwd_exec_t"))
print(ps.file_contexts("updpwd_exec_t"))

./admin/usermanage.fc:2:/bin/passwd		--	gen_context(system_u:object_r:passwd_exec_t,s0)
./admin/usermanage.fc:5:/usr/bin/chage		--	gen_context(system_u:object_r:passwd_exec_t,s0)
./admin/usermanage.fc:9:/usr/bin/passwd		--	gen_context(system_u:object_r:passwd_exec_t,s0)
./admin/usermanage.fc:10:/usr/bin/vigr		--	gen_context(system_u:object_r:admin_passwd_exec_t,s0)
./admin/usermanage.fc:11:/usr/bin/vipw		--	gen_context(system_u:object_r:admin_passwd_exec_t,s0)
./admin/usermanage.fc:21:/usr/sbin/grpconv	--	gen_context(system_u:object_r:admin_passwd_exec_t,s0)
./admin/usermanage.fc:22:/usr/sbin/grpunconv	--	gen_context(system_u:object_r:admin_passwd_exec_t,s0)
./admin/usermanage.fc:24:/usr/sbin/pwconv	--	gen_context(system_u:object_r:admin_passwd_exec_t,s0)
./admin/usermanage.fc:25:/usr/sbin/pwunconv	--	gen_context(system_u:object_r:admin_passwd_exec_t,s0)
./admin/usermanage.fc:29:/usr/sbin/vigr		--	gen_context(system_u:object_r:admin_passwd_exec_t,s0)
./admin/usermanage.fc:30:/usr/sbin/

Last thing - there is a userspace permission for changing _other_ users' passwords (which is what really matters here). I know that /bin/passwd checks this, but I'm not certain about things like /sbin/unix_update. But that's what this analysis is for - it tells us what code to go off an audit for trustworthiness.

So let's check the userspace permission for our login types.

In [17]:
p.terules_query(source=login_domains, tclass=["passwd"])

,source,target,tclass,perms,cond
0,secadm_t,secadm_t,passwd,"(passwd, rootok, chsh, chfn)",None
1,sysadm_t,sysadm_t,passwd,"(chfn, chsh, crontab, passwd, rootok)",None


Just what we would expect - `user_t` is not allowed to change other users' passwords.

So at the end of this things are basically what I would have expected with the exception of sandbox_domain (which I'm pretty sure is fine, but I don't understand well enough to know for sure).

# Reference Documentation

Some documentation on what's possible. This isn't exhaustive - mainly because it doesn't cover everything that Setools offers. One important note is that the policy object returned by `se.load_policy` is a subclass of the Setools policy object. All of the public methods from that class are available - you can see them at https://github.com/TresysTechnology/setools/blob/master/setools/policyrep/selinuxpolicy.py.


## Type and Attribute Searching
Find types by name

In [18]:
p.lookup_type("smbd_t")

Find types by regex

In [19]:
p.types_re("smbd")

The return from these functions is an object (even though it is rendered as a string here). You can, for example, show the attributes for a type by calling a method on the returned object.

In [20]:
sorted(p.lookup_type("smbd_t").attributes())

Notice that the output was sorted. This is both because it's nice for the output to be sorted, but also because most of the results from setools return generator functions instead of lists. That makes their output less convenient for use in jupyter notebook. For example, this is the output from the previous example not sorted.

In [21]:
p.lookup_type("smbd_t").attributes()

Because of this, we provide some convenience fucntions that simply make the output niecer. For example, find the attributes for a types:

In [22]:
p.attributes_for_type("smbd_t")

Find all of the types in an attribute:

In [38]:
p.types_in_attribute("files_unconfined_type")

[<Type(<SELinuxPolicy("fedora-25-policy.30")>, "abrt_handle_event_t")>,
 <Type(<SELinuxPolicy("fedora-25-policy.30")>, "anaconda_t")>,
 <Type(<SELinuxPolicy("fedora-25-policy.30")>, "authconfig_t")>,
 <Type(<SELinuxPolicy("fedora-25-policy.30")>, "bacula_unconfined_script_t")>,
 <Type(<SELinuxPolicy("fedora-25-policy.30")>, "boinc_project_t")>,
 <Type(<SELinuxPolicy("fedora-25-policy.30")>, "bootloader_t")>,
 <Type(<SELinuxPolicy("fedora-25-policy.30")>, "certmonger_unconfined_t")>,
 <Type(<SELinuxPolicy("fedora-25-policy.30")>, "cinder_api_t")>,
 <Type(<SELinuxPolicy("fedora-25-policy.30")>, "cinder_backup_t")>,
 <Type(<SELinuxPolicy("fedora-25-policy.30")>, "cinder_scheduler_t")>,
 <Type(<SELinuxPolicy("fedora-25-policy.30")>, "cinder_volume_t")>,
 <Type(<SELinuxPolicy("fedora-25-policy.30")>, "cloud_init_t")>,
 <Type(<SELinuxPolicy("fedora-25-policy.30")>, "cluster_t")>,
 <Type(<SELinuxPolicy("fedora-25-policy.30")>, "clvmd_t")>,
 <Type(<SELinuxPolicy("fedora-25-policy.30")>, "condo

Attributes by regex

In [24]:
p.attributes_re("unconfined")

[<TypeAttribute(<SELinuxPolicy("fedora-25-policy.30")>, "corenet_unconfined_type")>,
 <TypeAttribute(<SELinuxPolicy("fedora-25-policy.30")>, "dbusd_unconfined")>,
 <TypeAttribute(<SELinuxPolicy("fedora-25-policy.30")>, "devices_unconfined_type")>,
 <TypeAttribute(<SELinuxPolicy("fedora-25-policy.30")>, "files_unconfined_type")>,
 <TypeAttribute(<SELinuxPolicy("fedora-25-policy.30")>, "filesystem_unconfined_type")>,
 <TypeAttribute(<SELinuxPolicy("fedora-25-policy.30")>, "kern_unconfined")>,
 <TypeAttribute(<SELinuxPolicy("fedora-25-policy.30")>, "selinux_unconfined_type")>,
 <TypeAttribute(<SELinuxPolicy("fedora-25-policy.30")>, "sepgsql_unconfined_type")>,
 <TypeAttribute(<SELinuxPolicy("fedora-25-policy.30")>, "storage_unconfined_type")>,
 <TypeAttribute(<SELinuxPolicy("fedora-25-policy.30")>, "unconfined_domain_type")>,
 <TypeAttribute(<SELinuxPolicy("fedora-25-policy.30")>, "unconfined_login_domain")>,
 <TypeAttribute(<SELinuxPolicy("fedora-25-policy.30")>, "unconfined_usertype")>,

Lookup an attribute by name

In [25]:
p.lookup_typeattr("domain")

<TypeAttribute(<SELinuxPolicy("fedora-25-policy.30")>, "domain")>

Expand attribtutes in a list (this will be really long). The list can contain both types and attributes - it just returns the types unchanged into the output list.

In [26]:
p.expand_attributes([p.lookup_type("smbd_t"), p.lookup_typeattr("domain")])

{<Type(<SELinuxPolicy("fedora-25-policy.30")>, "NetworkManager_t")>,
 <Type(<SELinuxPolicy("fedora-25-policy.30")>, "abrt_dump_oops_t")>,
 <Type(<SELinuxPolicy("fedora-25-policy.30")>, "abrt_handle_event_t")>,
 <Type(<SELinuxPolicy("fedora-25-policy.30")>, "abrt_helper_t")>,
 <Type(<SELinuxPolicy("fedora-25-policy.30")>, "abrt_retrace_coredump_t")>,
 <Type(<SELinuxPolicy("fedora-25-policy.30")>, "abrt_retrace_worker_t")>,
 <Type(<SELinuxPolicy("fedora-25-policy.30")>, "abrt_t")>,
 <Type(<SELinuxPolicy("fedora-25-policy.30")>, "abrt_upload_watch_t")>,
 <Type(<SELinuxPolicy("fedora-25-policy.30")>, "abrt_watch_log_t")>,
 <Type(<SELinuxPolicy("fedora-25-policy.30")>, "accountsd_t")>,
 <Type(<SELinuxPolicy("fedora-25-policy.30")>, "acct_t")>,
 <Type(<SELinuxPolicy("fedora-25-policy.30")>, "admin_crontab_t")>,
 <Type(<SELinuxPolicy("fedora-25-policy.30")>, "afs_bosserver_t")>,
 <Type(<SELinuxPolicy("fedora-25-policy.30")>, "afs_fsserver_t")>,
 <Type(<SELinuxPolicy("fedora-25-policy.30")>, "

Lookup types or attributes from a list of strings.

In [27]:
p.lookup_type_or_attrs(["smbd_t", "domain"])

[<TypeAttribute(<SELinuxPolicy("fedora-25-policy.30")>, "domain")>,
 <Type(<SELinuxPolicy("fedora-25-policy.30")>, "smbd_t")>]

## Roles

Find roles - this is just a convenience wrapper around https://github.com/TresysTechnology/setools/blob/master/setools/rolequery.py.

In [28]:
p.roles_query(name="sysadm_r")

[<Role(<SELinuxPolicy("fedora-25-policy.30")>, "sysadm_r")>]

In [29]:
p.types_in_role("sysadm_r")

['NetworkManager_t',
 'admin_crontab_t',
 'admin_crontab_tmp_t',
 'aide_t',
 'alsa_home_t',
 'amanda_recover_t',
 'antivirus_home_t',
 'audio_home_t',
 'auditctl_t',
 'auth_home_t',
 'bacula_admin_t',
 'bootloader_t',
 'brctl_t',
 'cache_home_t',
 'cardmgr_t',
 'cdcc_t',
 'certwatch_t',
 'checkpc_t',
 'checkpolicy_t',
 'chfn_t',
 'chkpwd_t',
 'chrome_sandbox_home_t',
 'chrome_sandbox_nacl_t',
 'chrome_sandbox_t',
 'config_home_t',
 'container_t',
 'cronjob_t',
 'cvs_home_t',
 'data_home_t',
 'dbus_home_t',
 'dcc_client_t',
 'dcc_dbclean_t',
 'depmod_t',
 'dmidecode_t',
 'exim_t',
 'fetchmail_home_t',
 'firstboot_t',
 'fsadm_t',
 'gconf_home_t',
 'git_session_t',
 'git_user_content_t',
 'gkeyringd_gnome_home_t',
 'gnome_home_t',
 'gpg_secret_t',
 'groupadd_t',
 'gstreamer_home_t',
 'home_bin_t',
 'home_cert_t',
 'hostname_t',
 'httpd_helper_t',
 'httpd_user_content_t',
 'httpd_user_htaccess_t',
 'httpd_user_ra_content_t',
 'httpd_user_rw_content_t',
 'httpd_user_script_exec_t',
 'hwcloc

In [30]:
p.roles_for_type("smbd_t")

['system_r', 'unconfined_r']

## Rule Searching

These two methods are wrappers around an implementation that matches the API for Setools TERuleQuery, so the best documentation is at https://github.com/TresysTechnology/setools/blob/master/setools/terulequery.py.

One major API difference is that source and target paramaters can take a single type/attribute, string, or list.

The other difference is speed. This implementation is often 30x faster. It does fully pass the unit tests for the Setools implementation, so it is fast and API compliant. The speedup comes from the use of an index, so the first rule search after a policy is loaded will build the index (which can take a few seconds). Subsequent queries reuse the index.

In [31]:
p.terules_query(target="shadow_t", perms=se.file_w_perms)

,source,target,tclass,perms,cond
0,cockpit_session_t,shadow_t,file,"(append, unlink, open, write, setattr, create, getattr, ioctl, rename, read, link, lock)",None
1,files_unconfined_type,file_type,blk_file,"(unlink, execute, quotaon, swapon, link, getattr, append, ioctl, rename, audit_access, lock, write, setattr, read, relabelto, relabelfrom, create, execmod, open, mounton)",None
2,files_unconfined_type,file_type,chr_file,"(unlink, execute, quotaon, execute_no_trans, swapon, link, getattr, append, ioctl, rename, audit_access, lock, write, setattr, read, relabelto, relabelfrom, create, open, mounton)",None
3,files_unconfined_type,file_type,dir,"(unlink, execute, quotaon, swapon, link, getattr, append, ioctl, rename, add_name, open, rmdir, audit_access, lock, search, write, setattr, relabelto, mounton, relabelfrom, create, execmod, reparent, read, remove_name)",None
4,files_unconfined_type,file_type,fifo_file,"(unlink, execute, quotaon, swapon, link, getattr, append, ioctl, rename, audit_access, lock, write, setattr, read, relabelto, relabelfrom, create, execmod, open, mounton)",None
5,files_unconfined_type,file_type,file,"(unlink, execute, quotaon, execute_no_trans, swapon, link, getattr, append, ioctl, rename, audit_access, lock, write, setattr, read, relabelto, relabelfrom, create, open, mounton)",None
6,files_unconfined_type,file_type,filesystem,"(quotaget, mount, unmount, relabelto, relabelfrom, transition, quotamod, getattr, remount, associate)",None
7,files_unconfined_type,file_type,lnk_file,"(unlink, execute, quotaon, swapon, link, getattr, append, ioctl, rename, audit_access, lock, write, setattr, read, relabelto, relabelfrom, create, execmod, open, mounton)",None
8,files_unconfined_type,file_type,sock_file,"(unlink, execute, quotaon, swapon, link, getattr, append, ioctl, rename, audit_access, lock, write, setattr, read, relabelto, relabelfrom, create, execmod, open, mounton)",None
9,groupadd_t,shadow_t,file,"(append, unlink, open, write, setattr, relabelto, relabelfrom, create, getattr, ioctl, rename, read, link, lock)",None


In [32]:
# Search with a list for the target
p.terules_query(target=["ssh_home_t", "sshd_key_t"], perms=["write", "append"])

,source,target,tclass,perms,cond
0,abrt_dump_oops_t,non_security_file_type,dir,"(reparent, unlink, open, write, setattr, search, rmdir, add_name, create, getattr, ioctl, rename, read, link, lock, remove_name)",None
1,abrt_dump_oops_t,non_security_file_type,file,"(append, unlink, open, write, setattr, create, getattr, ioctl, rename, read, link, lock)",None
2,abrt_dump_oops_t,non_security_file_type,lnk_file,"(append, unlink, write, setattr, create, getattr, ioctl, rename, read, link, lock)",None
3,ajaxterm_ssh_t,ssh_home_t,dir,"(open, write, search, add_name, getattr, ioctl, read, lock, remove_name)",None
4,ajaxterm_ssh_t,ssh_home_t,file,"(append, unlink, open, write, setattr, create, getattr, ioctl, rename, read, link, lock)",None
5,ajaxterm_t,ssh_home_t,dir,"(open, write, search, add_name, getattr, ioctl, read, lock, remove_name)",None
6,ajaxterm_t,ssh_home_t,file,"(append, unlink, open, write, setattr, create, getattr, ioctl, rename, read, link, lock)",None
7,ajaxterm_t,ssh_home_t,lnk_file,"(append, unlink, write, setattr, create, getattr, ioctl, rename, read, link, lock)",None
8,ajaxterm_t,ssh_home_t,sock_file,"(append, unlink, open, write, setattr, create, getattr, ioctl, rename, read, link, lock)",None
9,automount_t,non_security_file_type,dir,"(reparent, unlink, open, write, setattr, search, rmdir, add_name, create, getattr, ioctl, rename, read, link, lock, remove_name)",None


In [33]:
p.transrules_query(source="initrc_t", default="smbd_t", tclass=["process"])

,source,target,tclass,default,filename,cond
0,initrc_t,smbd_exec_t,process,smbd_t,None,None


## Information Flow

The information flow queries allow you to focus more on the types and object classes without worrying so much about the details of the permissions. You can, instead, think in terms of read, write, or both.

For exampe, `domain_info_flow` shows all of the object types that a domain can read, write, or both.

In [34]:
# By default this shows writes.
p.domain_info_flow("smbd_t", tclass=["file"])

/home/karl/projects/st-policy-analysis/SPAN/span/span.py:171: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  df.style.applymap(dataframe_hide_none)


,Type,Conditional,Permissions
0,abrt_var_cache_t,None,"[append, getattr]"
1,afs_cache_t,None,"[read, write]"
2,auth_cache_t,None,"[append, create, getattr, ioctl, link, lock, open, read, rename, setattr, unlink, write]"
3,ctdbd_var_lib_t,None,"[append, create, getattr, ioctl, link, lock, open, read, rename, setattr, unlink, write]"
4,faillog_t,None,"[append, getattr, ioctl, lock, open, read, write]"
5,glusterd_var_lib_t,None,"[append, getattr, ioctl, lock, open, read, write]"
6,initrc_tmp_t,None,"[append, getattr, ioctl, lock, read, write]"
7,initrc_var_run_t,None,"[append, getattr, ioctl, lock, open, read, write]"
8,logfile,None,"[append, getattr, ioctl, lock]"
9,nmbd_var_run_t,None,"[append, getattr, ioctl, lock, open, read, write]"


In [35]:
# Show reads instead
p.domain_info_flow("smbd_t", tclass=["file"], direction="r")

/home/karl/projects/st-policy-analysis/SPAN/span/span.py:171: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  df.style.applymap(dataframe_hide_none)


,Type,Conditional,Permissions
0,abrt_helper_exec_t,None,"[execute, getattr, open, read]"
1,abrt_t,None,"[getattr, ioctl, lock, open, read]"
2,abrt_var_run_t,None,"[getattr, ioctl, lock, open, read]"
3,afs_cache_t,None,"[read, write]"
4,anon_inodefs_t,None,"[getattr, ioctl, lock, open, read]"
5,auth_cache_t,None,"[append, create, getattr, ioctl, link, lock, open, read, rename, setattr, unlink, write]"
6,base_ro_file_type,None,"[execute, execute_no_trans, getattr, ioctl, lock, open, read]"
7,bin_t,None,"[execute, execute_no_trans, getattr, ioctl, lock, open, read]"
8,cert_t,None,"[getattr, ioctl, lock, open, read]"
9,chkpwd_exec_t,None,"[execute, getattr, open, read]"


The information flow is weighted by bandwidth on a scale from 1 to 10. 10 would be something like `read` or `write`, while lower bandwidth permissions, like `getattr` would be lower.

Here we set the minimum weight lower and show the additional types that returns.

In [39]:
set(p.domain_info_flow("NetworkManager_t", min_weight=1).Type) - set(p.domain_info_flow("NetworkManager_t").Type)

/home/karl/projects/st-policy-analysis/SPAN/span/span.py:171: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  df.style.applymap(dataframe_hide_none)
/home/karl/projects/st-policy-analysis/SPAN/span/span.py:171: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  df.style.applymap(dataframe_hide_none)


{<span.span.Type at 0x7f54574c5850>,
 <span.span.Type at 0x7f54574c5d10>}

You can also look from the other direction - the perspective of the object.

In [ ]:
p.object_info_flow("bin_t")

The concept of read and write works for non-file-like object classes as well.

In [ ]:
p.domain_info_flow("smbd_t", tclass=["packet"])

You can see which permissions are included with `info_flow_perms`.

In [ ]:
p.info_flow_perms(tclass=["dir"], min_weight=1)

In [ ]:
p.info_flow_perms(tclass=["dir"], min_weight=10)

## Summaries

These are a quick way to gather related information about something in the policy.

In [ ]:
p.types_summary(p.types_re("smb"))

In [ ]:
p.domain_summary("httpd_t")

In [ ]:
p.attribute_summary("domain")

In [ ]:
p.file_summary("bin_t")

In [ ]:
p.packet_summary("dns_client_packet_t")

# Policy Source

Find a type definition

In [ ]:
# note the use of print to make this look nice
print(ps.type_def("kernel_t"))

In [ ]:
print(ps.attr_def("domain"))

In [ ]:
ps.genfscon("selinuxfs")

In [ ]:
print(ps.file_contexts("httpd_exec_t"))

Search for rules (this is just grep really)

In [ ]:
print(ps.rules_search("allow sshd_t"))

Show an entire module

In [ ]:
print(ps.get_module("services/ssh.te"))